# Elastic Search

## Install Dependency

In [2]:
%pip install elasticsearch

     -------------------------------------- 543.1/543.1 KB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 64.4/64.4 KB ? eta 0:00:00
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LENOVO\Documents\wiwie\projects\ai-sqlite-and-elastic-search-exercise\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Import Library

In [4]:
from elasticsearch import Elasticsearch
import time
import json
import pandas as pd

## Read Dataset

In [5]:
dataset_csv = pd.read_csv("../datasets/data.csv", encoding='ISO-8859-1')

dataset_csv.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Preprocessing Dataset

In [6]:
dataset_csv = dataset_csv.dropna()

In [7]:
dataset_csv.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## Connect to Elastic Search

In [8]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

## Create Index and Insert Data

In [11]:
response = es.options(ignore_status=[400]).indices.create(index='e_commerce_sales_order')
print(json.dumps(response.body, indent=4))

{
    "acknowledged": true,
    "shards_acknowledged": true,
    "index": "e_commerce_sales_order"
}


In [10]:
response = es.options(ignore_status=[400, 404]).indices.delete(index='e_commerce_sales_order')
print(json.dumps(response.body, indent=4))

{
    "acknowledged": true
}


In [13]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from utils.elastic_search_utils import df_to_elasticsearch
from elasticsearch.helpers import bulk

bulk(es, df_to_elasticsearch(dataset_csv, 'e_commerce_sales_order'))

(406829, [])

In [14]:
response = es.search(index='e_commerce_sales_order', body={
    'size': 1,
    'from': 0
})
print(json.dumps(response.body, indent=4))

{
    "took": 1247,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 10000,
            "relation": "gte"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "e_commerce_sales_order",
                "_id": "756d7638-f0bb-48eb-8fc0-272cff8392b0",
                "_score": 1.0,
                "_source": {
                    "InvoiceNo": "536365",
                    "StockCode": "85123A",
                    "Description": "WHITE HANGING HEART T-LIGHT HOLDER",
                    "Quantity": 6,
                    "InvoiceDate": "12/1/2010 8:26",
                    "UnitPrice": 2.55,
                    "CustomerID": 17850.0,
                    "Country": "United Kingdom"
                }
            }
        ]
    }
}


In [16]:
from elasticsearch import Elasticsearch

# Connect to the Elasticsearch server
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

index_name = 'e_commerce_sales_order' #change the my_index_name value

# Get index mapping (equivalent to getting column names in SQL)
mapping = es.indices.get_mapping(index=index_name)
es_columns = list(mapping[index_name]['mappings']['properties'].keys()) #the variable es_columns is used for submission, do not change the variable name
print(f"Columns: {es_columns}")

# Count documents in the index (equivalent to counting rows in SQL)
es_num_rows = es.count(index=index_name)['count'] #the variable es_num_rows is used for submission, do not change the variable name
print(f"Number of rows: {es_num_rows}")


Columns: ['Country', 'CustomerID', 'Description', 'InvoiceDate', 'InvoiceNo', 'Quantity', 'StockCode', 'UnitPrice']
Number of rows: 406829


## TASK-01: All Transactions for a Specific Country (Germany)

In [17]:
index_name = 'e_commerce_sales_order'

query = {
    "query": {
        "match": {
            "Country": "Germany"  
        }
    }
}

response = es.count(index=index_name, body=query)

es_transactions_in_germany = response['count']
print(f"Number of documents matching the query: {es_transactions_in_germany}")

Number of documents matching the query: 9495


## TASK-02: Unique Products in Germany

In [18]:
index_name = 'e_commerce_sales_order'

country = "Germany"  
query = {
    "size": 0,  
    "query": {
        "match": {
            "Country": country
        }
    },
    "aggs": {
        "unique_products": {
            "cardinality": {
                "field": "Description.keyword"  # Field representing the product (use `.keyword` for exact match)
            }
        }
    }
}

response = es.search(index=index_name, body=query)

es_total_unique_products = response['aggregations']['unique_products']['value']
print(f"Unique products in {country}: {es_total_unique_products}")

Unique products in Germany: 1703


## TASK-03: Top 5 Most Purchased Products in Germany

In [19]:
index_name = 'e_commerce_sales_order'

# Define the query to group by stock ID and sum product quantities where country = Germany
query = {
    "size": 0,  # No need to return actual documents
    "query": {
        "match": {
            "Country": "Germany"  # Filter for Germany (use .keyword for exact match)
        }
    },
    "aggs": {
        "group_by_description": {
            "terms": {
                "field": "Description.keyword",  # Group by Stock ID (or StockCode)
                "size": 5,
                "order": {  # Sort by total quantity in ascending order
                    "total_quantity.value": "desc"
                },
            },
            "aggs": {
                "total_quantity": {
                    "value_count": {
                        "field": "Description.keyword"  # Sum the quantity for each stock ID
                    }
                }
            }
        }
    }
}

# Execute the query
response = es.search(index=index_name, body=query)

es_top_products = {}
# Extract and print the results
buckets = response['aggregations']['group_by_description']['buckets']
for bucket in buckets:
    stock_code = bucket['key']
    total_quantity = bucket['total_quantity']['value']
    es_top_products[stock_code] = int(total_quantity)
    print(f"Stock ID: {stock_code}, Total Quantity: {total_quantity}")
    
es_top_products

Stock ID: POSTAGE, Total Quantity: 383
Stock ID: ROUND SNACK BOXES SET OF4 WOODLAND , Total Quantity: 120
Stock ID: REGENCY CAKESTAND 3 TIER, Total Quantity: 81
Stock ID: ROUND SNACK BOXES SET OF 4 FRUITS , Total Quantity: 78
Stock ID: PLASTERS IN TIN WOODLAND ANIMALS, Total Quantity: 67


{'POSTAGE': 383,
 'ROUND SNACK BOXES SET OF4 WOODLAND ': 120,
 'REGENCY CAKESTAND 3 TIER': 81,
 'ROUND SNACK BOXES SET OF 4 FRUITS ': 78,
 'PLASTERS IN TIN WOODLAND ANIMALS': 67}

## Submission

In [ ]:
student_id = "REAINTCZ" 
name = "Wiwie Sanjaya"
drive_link = "https://github.com/wiwiewei18/sqlite-and-elastic-search-exercise"  

assignment_id = "00_database_project"

from rggrader import submit, submit_image

# question_id = "09_es_columns"
# submit(student_id, name, assignment_id, str(es_columns), question_id, drive_link)
# question_id = "10_es_num_rows"
# submit(student_id, name, assignment_id, str(es_num_rows), question_id, drive_link)
# question_id = "11_es_total_unique_products"
# submit(student_id, name, assignment_id, str(es_total_unique_products), question_id, drive_link)
# question_id = "12_es_transactions_in_germany"
# submit(student_id, name, assignment_id, str(es_transactions_in_germany), question_id, drive_link)
# question_id = "13_es_top_products"
# submit(student_id, name, assignment_id, str(es_top_products), question_id, drive_link)

'Assignment successfully submitted'